In [ ]:
import pandas as pd
import os
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm
import time
from rasterstats import zonal_stats
from scipy.stats import pearsonr
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import raster_func  #import function python filee

boundary = gpd.read_file('../Karnataka/District.geojson')

In [ ]:
fig, df = raster_func.correlation('NDVI/Karnataka/', 2012, 2022,  boundary, 'NDVI for KA districts')